# Drowsiness Detection OpenCV


This code can detect your eyes and alert when the user is drowsy.

## Applications
This can be used by riders who tend to drive for a longer period of time that may lead to accidents.

### Algorithm

Each eye is represented by 6 (x, y)-coordinates, starting at the left-corner of the eye (as if you were looking at the person), and then working clockwise around the eye:.

<img src="eye1.jpg">

### Condition

It checks 20 consecutive frames and if the Eye Aspect ratio is lesst than 0.25, Alert is generated.

#### Relationship

<img src="eye2.png">

#### Summing up

<img src="eye3.jpg">

jupyter kernel by Manuel Romero (mrm8488@gmail.com or @mrm8488)

In [1]:
import os
import glob
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2

print("✅ Setup complete!")

✅ Setup complete!


In [2]:
# CELL 2: VERIFIKASI DATASET
# ===========================
def check_dataset():
    dataset_path = "mobilephone.v7-v7.yolov5pytorch"
    
    if not os.path.exists(dataset_path):
        print(f"❌ Dataset folder '{dataset_path}' not found!")
        print("Make sure you extracted the dataset in the same directory as this notebook")
        return False
    
    # Check structure
    required_items = ['train', 'valid', 'data.yaml']
    for item in required_items:
        if os.path.exists(os.path.join(dataset_path, item)):
            print(f"✅ Found: {item}")
        else:
            print(f"❌ Missing: {item}")
            return False
    
    # Count images
    train_images = len(glob.glob(f"{dataset_path}/train/images/*.jpg")) + len(glob.glob(f"{dataset_path}/train/images/*.png"))
    valid_images = len(glob.glob(f"{dataset_path}/valid/images/*.jpg")) + len(glob.glob(f"{dataset_path}/valid/images/*.png"))
    
    print(f"📊 Dataset Info:")
    print(f"   Training images: {train_images}")
    print(f"   Validation images: {valid_images}")
    print(f"   Total: {train_images + valid_images}")
    
    return True

# Jalankan pengecekan
if check_dataset():
    print("🎉 Dataset ready for training!")
else:
    print("⚠️  Please fix dataset issues before continuing")

✅ Found: train
✅ Found: valid
✅ Found: data.yaml
📊 Dataset Info:
   Training images: 9009
   Validation images: 326
   Total: 9335
🎉 Dataset ready for training!


In [ ]:
# CELL 3: TRAINING MODEL
# =======================
def train_phone_model():
    """Training model mobile phone detection"""
    
    print("🚀 Starting training...")
    print("⏱️  Estimated time: 15-45 minutes")
    print("📝 You can monitor progress in the output below")
    
    # Load pre-trained YOLOv8 model
    model = YOLO('yolov8n.pt')  # Will download automatically
    
    # Start training
    results = model.train(
        data='mobilephone.v7-v7.yolov5pytorch/data.yaml',  # Path to your dataset
        epochs=50,              # Number of training epochs
        imgsz=640,              # Image size
        batch=8,               # Batch size (reduce to 8 if memory error)
        device=0,               # Use GPU (change to 'cpu' if no GPU)
        project='mobile_phone_training',  # Project folder name
        name='phone_detector',  # Experiment name
        patience=15,            # Early stopping patience
        save=True,              # Save checkpoints
        cache=True,             # Cache images for faster training
        augment=True,           # Data augmentation
        verbose=True            # Show detailed output
    )
    
    model_path = 'mobile_phone_training/phone_detector/weights/best.pt'
    print(f"🎉 Training completed!")
    print(f"📦 Model saved at: {model_path}")
    
    return model_path

# UNCOMMENT LINE BERIKUT UNTUK MULAI TRAINING:
trained_model_path = train_phone_model()


🚀 Starting training...
⏱️  Estimated time: 15-45 minutes
📝 You can monitor progress in the output below
Ultralytics 8.3.162  Python-3.8.8 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=mobilephone.v7-v7.yolov5pytorch/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=phone_detector6, nbs=64, nms=Fa

train: Scanning D:\Codingan Project Skripsi Jaka\mobilephone.v7-v7.yolov5pytorch\train\labels.cache... 9009 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9009/9009 [00:00<?, ?it/s]


WARNING train: 11.3GB RAM required to cache images with 50% safety margin but only 5.8/13.9GB available, not caching images
val: Fast image access  (ping: 0.10.1 ms, read: 115.731.8 MB/s, size: 71.0 KB)


val: Scanning D:\Codingan Project Skripsi Jaka\mobilephone.v7-v7.yolov5pytorch\valid\labels.cache... 326 images, 0 backgrounds, 0 corrupt: 100%|██████████| 326/326 [00:00<?, ?it/s]


WARNING cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.3GB RAM): 100%|██████████| 326/326 [00:00<00:00, 405.38it/s]


Plotting labels to mobile_phone_training\phone_detector6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to mobile_phone_training\phone_detector6
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         2G      1.072      1.557      1.344         27        640:  38%|███▊      | 430/1127 [02:45<02:56,  3.94it/s]

In [ ]:
# CELL 4: TEST MODEL
# ==================
def test_model():
    """Test model yang sudah ditraining"""
    
    model_path = 'mobile_phone_training/phone_detector5/weights/best.pt'
    
    # Check if model exists
    if not os.path.exists(model_path):
        print(f"❌ Model not found at: {model_path}")
        print("Please run training first!")
        return
    
    # Load model
    model = YOLO(model_path)
    print(f"✅ Model loaded from: {model_path}")
    
    # Test dengan beberapa gambar validasi
    val_images = glob.glob("mobilephone-v7-v7.yolov5pytorch/valid/images/*.jpg")[:4]
    
    if len(val_images) == 0:
        print("❌ No validation images found!")
        return
    
    # Run inference dan tampilkan hasil
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(val_images):
        # Predict
        results = model(img_path, conf=0.3)
        
        # Get annotated image
        annotated_img = results[0].plot()
        annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(annotated_img)
        axes[idx].set_title(f"Test {idx+1}")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
    print("✅ Model testing completed!")

# UNCOMMENT SETELAH TRAINING SELESAI:
test_model()

✅ Model loaded from: mobile_phone_training/phone_detector5/weights/best.pt
❌ No validation images found!


In [ ]:
# CELL 5: VALIDASI PERFORMA
# ==========================
def validate_model():
    """Cek performa model"""
    
    model_path = 'mobile_phone_training/phone_detector5/weights/best.pt'
    
    if not os.path.exists(model_path):
        print("❌ Model not found! Train the model first.")
        return
    
    model = YOLO(model_path)
    
    # Run validation
    print("🔍 Running validation...")
    metrics = model.val()
    
    print("📊 Model Performance:")
    print(f"   mAP@0.5: {metrics.box.map50:.3f}")      # Mean Average Precision at IoU 0.5
    print(f"   mAP@0.5:0.95: {metrics.box.map:.3f}")   # Mean Average Precision at IoU 0.5-0.95
    print(f"   Precision: {metrics.box.mp:.3f}")        # Precision
    print(f"   Recall: {metrics.box.mr:.3f}")           # Recall
    
    if metrics.box.map50 > 0.8:
        print("🎉 Excellent model performance!")
    elif metrics.box.map50 > 0.6:
        print("✅ Good model performance!")
    elif metrics.box.map50 > 0.4:
        print("⚠️  Fair performance - consider more training")
    else:
        print("❌ Poor performance - check dataset or train longer")
    
    return metrics

# UNCOMMENT SETELAH TRAINING:
validation_metrics = validate_model()

🔍 Running validation...
Ultralytics 8.3.162  Python-3.8.8 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.30.1 ms, read: 10.33.3 MB/s, size: 72.2 KB)


val: Scanning D:\Codingan Project Skripsi Jaka\mobilephone.v7-v7.yolov5pytorch\valid\labels.cache... 326 images, 0 backgrounds, 0 corrupt: 100%|██████████| 326/326 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:03<00:00,  5.72it/s]


                   all        326        451      0.955      0.901      0.955      0.826
Speed: 0.5ms preprocess, 5.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\detect\val
📊 Model Performance:
   mAP@0.5: 0.955
   mAP@0.5:0.95: 0.826
   Precision: 0.955
   Recall: 0.901
🎉 Excellent model performance!


In [ ]:
# CELL 6: LIVE WEBCAM TEST
# =========================
def test_with_webcam():
    """Test model dengan webcam langsung"""
    
    model_path = 'mobile_phone_training/phone_detector5/weights/best.pt'
    
    if not os.path.exists(model_path):
        print("❌ Model not found!")
        return
    
    model = YOLO(model_path)
    cap = cv2.VideoCapture(0)
    
    print("📹 Testing with webcam...")
    print("Hold a phone in front of the camera")
    print("Press 'q' to quit, 's' for screenshot")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run detection
        results = model(frame, conf=0.3)
        
        # Draw results
        annotated_frame = results[0].plot()
        
        # Display
        cv2.imshow('Phone Detection Test', annotated_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            cv2.imwrite('phone_detection_test.jpg', annotated_frame)
            print("📸 Screenshot saved!")
    
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Webcam test completed!")

# UNCOMMENT UNTUK TEST WEBCAM:
test_with_webcam()

📹 Testing with webcam...
Hold a phone in front of the camera
Press 'q' to quit, 's' for screenshot

0: 480x640 1 Mobile-phone, 9.3ms
Speed: 1.9ms preprocess, 9.3ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Mobile-phone, 9.2ms
Speed: 2.9ms preprocess, 9.2ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.5ms
Speed: 1.5ms preprocess, 9.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.2ms
Speed: 1.7ms preprocess, 9.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.3ms
Speed: 1.6ms preprocess, 9.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.1ms
Speed: 1.3ms preprocess, 9.1ms inference, 0.8ms postprocess per image at shape (1,

In [ ]:
# CELL 7: FINAL CHECK & INTEGRATION
# ==================================
def final_check():
    """Final check sebelum integrasi"""
    
    model_path = 'mobile_phone_training/phone_detector5/weights/best.pt'
    
    print("📋 FINAL CHECKLIST:")
    
    # Check model file
    if os.path.exists(model_path):
        print(f"✅ Trained model: {model_path}")
        
        # Check model size
        size_mb = os.path.getsize(model_path) / (1024*1024)
        print(f"   Size: {size_mb:.1f} MB")
        
        # Quick test
        try:
            model = YOLO(model_path)
            print(f"   Classes: {model.names}")
            print("✅ Model loads successfully!")
        except Exception as e:
            print(f"❌ Model load error: {e}")
    else:
        print(f"❌ Model file missing: {model_path}")
        return False
    
    print("\n🎯 READY FOR INTEGRATION!")
    print("Copy this model path to your drowsiness detection notebook:")
    print(f"'{model_path}'")
    
    return True

# Jalankan final check
final_check()

# NEXT STEPS UNTUK INTEGRASI
# ===========================
print("\n" + "="*50)
print("🔄 NEXT: INTEGRATE TO DROWSINESS DETECTION")
print("="*50)
print("1. Copy model path: 'mobile_phone_training/phone_detector/weights/best.pt'")
print("2. In your drowsiness notebook, change:")
print("   OLD: model = YOLO('yolov5/runs/train/hp_detector13/weights/best.pt')")
print("   NEW: phone_model = YOLO('mobile_phone_training/phone_detector/weights/best.pt')")
print("3. Update detection loop to use phone_model")
print("4. Adjust confidence threshold if needed (0.3-0.5)")
print("="*50)

# TROUBLESHOOTING
# ===============
print("\n🔧 TROUBLESHOOTING:")
print("- Training fails with memory error: Reduce batch size to 8")
print("- No GPU detected: Change device=0 to device='cpu'")
print("- Poor detection: Train longer (epochs=100) or check dataset quality")
print("- Webcam not working: Try VideoCapture(1) or VideoCapture(2)")
print("- False detections: Increase confidence threshold to 0.4-0.5")

📋 FINAL CHECKLIST:
✅ Trained model: mobile_phone_training/phone_detector5/weights/best.pt
   Size: 5.9 MB
   Classes: {0: 'Mobile-phone'}
✅ Model loads successfully!

🎯 READY FOR INTEGRATION!
Copy this model path to your drowsiness detection notebook:
'mobile_phone_training/phone_detector5/weights/best.pt'

🔄 NEXT: INTEGRATE TO DROWSINESS DETECTION
1. Copy model path: 'mobile_phone_training/phone_detector/weights/best.pt'
2. In your drowsiness notebook, change:
   OLD: model = YOLO('yolov5/runs/train/hp_detector13/weights/best.pt')
   NEW: phone_model = YOLO('mobile_phone_training/phone_detector/weights/best.pt')
3. Update detection loop to use phone_model
4. Adjust confidence threshold if needed (0.3-0.5)

🔧 TROUBLESHOOTING:
- Training fails with memory error: Reduce batch size to 8
- No GPU detected: Change device=0 to device='cpu'
- Poor detection: Train longer (epochs=100) or check dataset quality
- Webcam not working: Try VideoCapture(1) or VideoCapture(2)
- False detections: Incr